<a href="https://colab.research.google.com/github/WALE-ONI/ADS-ASSIGNMENT-2-STATISTICS-AND-TREND-/blob/main/PySpark_Assignment_Sep23_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ADS2 - Assignment 1 - Data Handling and Processing with PySpark

**STUDENT NAME - OLAJIDE STEPHEN WALE-ONI**

**STUDENT ID - 22013012**

In this assignment, you will be analysing the popularity of films and TV shows on the streaming platform, Netflix. Using your knowledge of PySpark DataFrames and Spark SQL, you will produce a number of "downstream" data products to analyse trends in global streaming habits.

Download the dataset from this [Kaggle](https://www.kaggle.com/dhruvildave/netflix-top-10-tv-shows-and-films) page. A copy of the `all_weeks_countries.csv` file is also available on the canvas page for this assignment.

Your task is to load in the data and produce a number of "downstream" data products and plots as described below.

The PySpark installation and setup is provided below for conveinience.

IMPORTANT: DO NOT EDIT OR REMOVE THE COMMENT TAGS IN THE CODE CELLS

In [177]:
# CodeGrade Tag Init1

# Apache Spark uses Java, so first we must install that
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [178]:
# CodeGrade Tag Init2
# Mount Google Drive and unpack Spark
from google.colab import drive
drive.mount('/content/drive')
!tar xzf /content/drive/MyDrive/spark-3.3.0-bin-hadoop3.tgz

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [179]:
# CodeGrade Tag Init3
# Set up environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

In [180]:
# CodeGrade Tag Init4
# Install findspark, which helps python locate the pyspark module files
!pip install -q findspark
import findspark
findspark.init()

In [181]:
# Finally, we initialse a "SparkSession", which handles the computations
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [182]:
# Load the all_weeks_countries.csv into your Colab Notebook as a DataFrame.
netflixcsvpath = '/content/drive/MyDrive/all-weeks-countries.csv'

# Data is loaded with header: True and an inferred schema
netflixDF = (spark
           .read
           .option('header', 'True')
           .option('inferSchema', 'True')
           .csv(netflixcsvpath)
          )

In [183]:
# pyspark.sql.functions countains all the transformations and actions you will
# need
from pyspark.sql import functions as F

# Exercise 1 - Data Preparation


1.   Create two separate DataFrames for Films and TV.
2.   For the Films data, drop the column containing the season names.
3.   For the TV data, replace any null values in the season name column with the show name.

In [184]:
# CodeGrade Tag Ex1a

### Display the table and its schema

##SOLUTION 1a

netflixDF.show(truncate = False)# the truncate function is to make sure all text in each colums are shown

#display table and schema
netflixDF.show()
netflixDF.printSchema()

+------------+------------+-------------------+--------+-----------+----------------------------------+------------------------------------+--------------------------+
|country_name|country_iso2|week               |category|weekly_rank|show_title                        |season_title                        |cumulative_weeks_in_top_10|
+------------+------------+-------------------+--------+-----------+----------------------------------+------------------------------------+--------------------------+
|Argentina   |AR          |2022-08-21 00:00:00|Films   |1          |Look Both Ways                    |null                                |1                         |
|Argentina   |AR          |2022-08-21 00:00:00|Films   |2          |Day Shift                         |null                                |2                         |
|Argentina   |AR          |2022-08-21 00:00:00|Films   |3          |Bank Robbers: The Last Great Heist|null                                |2                   

In [185]:
# CodeGrade Tag Ex1b

### Seperate the data into two DataFrames for Films and TV
### Call the dataframes tvDF and filmsDF


##SOLUTION 1b

#seperating the data into  filmsDF and tvDF
filmsDF = netflixDF.filter(netflixDF["category"] == "Films")
tvDF = netflixDF.filter(netflixDF["category"] == "TV")

#calling tvDF and filmsDF
tvDF.show()
filmsDF.show()

+------------+------------+-------------------+--------+-----------+--------------------+--------------------+--------------------------+
|country_name|country_iso2|               week|category|weekly_rank|          show_title|        season_title|cumulative_weeks_in_top_10|
+------------+------------+-------------------+--------+-----------+--------------------+--------------------+--------------------------+
|   Argentina|          AR|2022-08-21 00:00:00|      TV|          1| Pasión de Gavilanes|Pasión de Gavilan...|                         5|
|   Argentina|          AR|2022-08-21 00:00:00|      TV|          2|        Another Self|Another Self: Sea...|                         4|
|   Argentina|          AR|2022-08-21 00:00:00|      TV|          3| Pasión de Gavilanes|Pasión de Gavilan...|                        58|
|   Argentina|          AR|2022-08-21 00:00:00|      TV|          4|            Manifest|  Manifest: Season 1|                         6|
|   Argentina|          AR|2022-08

In [186]:
# CodeGrade Tag Ex1c

### Drop the 'season_title' column from the Films DataFrame, display the table

##SOLUTION 1c
#dropping 'season_title column
filmsDF= filmsDF.drop("season_title")
filmsDF.show()


+------------+------------+-------------------+--------+-----------+--------------------+--------------------------+
|country_name|country_iso2|               week|category|weekly_rank|          show_title|cumulative_weeks_in_top_10|
+------------+------------+-------------------+--------+-----------+--------------------+--------------------------+
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          1|      Look Both Ways|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          2|           Day Shift|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          3|Bank Robbers: The...|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          4|   The Next 365 Days|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          5|The Angry Birds M...|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|        

In [187]:
# CodeGrade Tag Ex1d

### Use the F.isnull function to create a column showing where there are null
### values in the 'season_title' column. Replace the null values with the
### corresponding value from the 'show_title' column, then replace the
### 'season_title' column in the tvDF DataFrame.

##SOLUTION 1d
# Create a new column 'empty_column' to flag the null values in 'season_title'
null_values = netflixDF.withColumn('null_season_title', F.isnull(F.col('season_title')))

# To Replace null values in 'season_title' with the value from 'show_title'
replace_null_values= null_values.withColumn('season_title', F.when(null_values['null_season_title'], null_values['show_title']).otherwise(null_values['season_title']))


# Replace null values in 'season_title' with values from 'show_title' in the tvDF DataFrame
#tvDF_replace_values = tvDF.withColumn('season_title', F.coalesce(F.col('season_title'), F.col('show_title')))


tvDF_replace_values = tvDF.withColumn(
    'season_title',
    F.coalesce(tvDF['season_title'], tvDF['show_title'])
)
# Show the resulting DataFrames
null_values.show()
replace_null_values.show()
tvDF_replace_values.show()

+------------+------------+-------------------+--------+-----------+--------------------+--------------------+--------------------------+-----------------+
|country_name|country_iso2|               week|category|weekly_rank|          show_title|        season_title|cumulative_weeks_in_top_10|null_season_title|
+------------+------------+-------------------+--------+-----------+--------------------+--------------------+--------------------------+-----------------+
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          1|      Look Both Ways|                null|                         1|             true|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          2|           Day Shift|                null|                         2|             true|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          3|Bank Robbers: The...|                null|                         2|             true|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|        

# Exercise 2

Making use of the "groupBy" and "where" methods, find the number of weeks the show "Stranger Things" was in the Top 10 for the United Kingdom across all seasons. Store your result in a variable named "STWeeks"


In [188]:
# CodeGrade Tag Ex2
### Use the "where" method to create a new dataframe containing the data for
### the show Stranger Things in the Uniter Kingdom. Call this dataframe STDF.


### Using "groupBy" method and "F.count_distinct" function, find the total number of weeks
### Stranger Things spent in the top 10 of the UK, across all seasons. Show the
### result.

##SOLUTION 2


# Filter the DataFrame for "Stranger Things" in the UK and rank in top 10
STDF = netflixDF.where(
    (netflixDF['show_title'] == 'Stranger Things') &
    (netflixDF['country_name'] == 'United Kingdom')
)

# Aggregate to find the maximum value of 'cumulative_weeks_in_top_10'
# Group by 'country_name' and 'show_title' to get distinct rows for each country and show combination
# Even though the 'country_name' and 'show_title' are constant in this case, they are included to show the structure clearly
#weeks_for_top_10 = STDF.groupBy('show_title').agg(F.count_distinct('week').alias('max_cumulative_weeks_in_top_10')
#)

weeks_for_top_10 = STDF.groupBy('show_title').agg(
    F.count_distinct('week').alias('max_cumulative_weeks_in_top_10')
)

# Show the filtered DataFrame for "Stranger Things" in the top 10 UK
STDF.show()

# Show the aggregated result with the maximum cumulative weeks in top 10 for "Stranger Things" in the UK
weeks_for_top_10.show()






+--------------+------------+-------------------+--------+-----------+---------------+-----------------+--------------------------+
|  country_name|country_iso2|               week|category|weekly_rank|     show_title|     season_title|cumulative_weeks_in_top_10|
+--------------+------------+-------------------+--------+-----------+---------------+-----------------+--------------------------+
|United Kingdom|          GB|2022-08-21 00:00:00|      TV|          3|Stranger Things|Stranger Things 4|                        13|
|United Kingdom|          GB|2022-08-14 00:00:00|      TV|          3|Stranger Things|Stranger Things 4|                        12|
|United Kingdom|          GB|2022-08-14 00:00:00|      TV|         10|Stranger Things|Stranger Things 3|                        12|
|United Kingdom|          GB|2022-08-07 00:00:00|      TV|          2|Stranger Things|Stranger Things 4|                        11|
|United Kingdom|          GB|2022-08-07 00:00:00|      TV|          9|Strang

# Exercise 3

Produce a dataframe containing only the Top 25 TV seasons in the UK, based on the number of weeks they spent in the Top 10.

In [189]:
# CodeGrade Tag Ex3
### Produce a dataframe containing the top 25 seasons by number of weeks in the
### top 10 of the United Kingdom, sorted by number of weeks. Store the dataframe
### in a variable called Top25


##SOLUTION 3
# Filter the DataFrame for entries in the United Kingdom and in the top 10
uk_top10 = netflixDF.where(
    (netflixDF['country_name'] == 'United Kingdom') &
    (netflixDF['weekly_rank'] <= 10)
)
Top_25= netflixDF.groupBy('country_name', 'category', 'season_title').agg(F.max(netflixDF['cumulative_weeks_in_top_10']).
                                                                  alias('cumulative_weeks_in_top_10')) # grouping by each elements

Top_25.show()


+--------------+--------+--------------------+--------------------------+
|  country_name|category|        season_title|cumulative_weeks_in_top_10|
+--------------+--------+--------------------+--------------------------+
|       Austria|      TV|First Kill: Season 1|                         3|
|       Bahamas|      TV|The Ultimatum: Ma...|                         4|
|       Bahrain|      TV|Too Hot to Handle...|                         2|
|       Bahrain|      TV|Squid Game: Season 1|                        16|
|       Bahrain|      TV|Turning Point: 9/...|                         2|
|    Bangladesh|      TV|Locke & Key: Seas...|                         2|
|       Bolivia|      TV|The Cuphead Show!...|                         3|
|        Brazil|      TV| Riverdale: Season 5|                         2|
|      Bulgaria|      TV|Dark Desire: Seas...|                         2|
|      Bulgaria|      TV| Mr. Robot: Season 1|                         2|
|        Canada|      TV|PAW Patrol: S

# Exercise 4

For the show "Young Sheldon", find the country where each season spent the most time in the Top 10.

In [190]:
# CodeGrade Tag Ex4
### For each season of the show "Young Sheldon" find the countries where it spent
### the most time in the Top 10

# Filter the DataFrame for the show "Young Sheldon"
young_sheldon_df = netflixDF.filter(netflixDF['show_title'] == 'Young Sheldon')

# Group by 'season_title' and 'country_name', and find the max 'cumulative_weeks_in_top_10' for each combination
country_season_max_weeks = young_sheldon_df.groupBy('season_title', 'country_name').agg(
    F.max('cumulative_weeks_in_top_10').alias('max_weeks_in_top_10')
)

# Now find the maximum 'cumulative_weeks_in_top_10' for each season across all countries
max_weeks_per_season = country_season_max_weeks.groupBy('season_title').agg(
    F.max('max_weeks_in_top_10').alias('max_weeks_in_top_10')
)

# Join the max weeks per season back to the country-season max weeks to find the corresponding countries
top_countries_per_season = country_season_max_weeks.join(
    max_weeks_per_season,
    on=['season_title', 'max_weeks_in_top_10'],
    how='inner'
)

# Select and display the desired columns
top_countries_per_season.select('country_name', 'season_title', 'max_weeks_in_top_10').show(truncate=False)


+--------------+-------------------------+-------------------+
|country_name  |season_title             |max_weeks_in_top_10|
+--------------+-------------------------+-------------------+
|Australia     |Young Sheldon: Season 4  |1                  |
|New Zealand   |Young Sheldon: Season 4  |1                  |
|United Kingdom|Young Sheldon: : Season 2|6                  |
|Ireland       |Young Sheldon: : Season 1|6                  |
|United Kingdom|Young Sheldon: : Season 1|6                  |
|Canada        |Young Sheldon: : Season 1|6                  |
|United Kingdom|Young Sheldon: : Season 3|4                  |
+--------------+-------------------------+-------------------+



# Exercise 5

For each country, find the film that spent the most time in the Top 10.

In [191]:
# CodeGrade Tag Ex5
### For each country, find the film that spent the most time in the Top 10
### Display the results in a Dataframe ordered by country name.


# CodeGrade Tag Ex5
### For each country, find the film that spent the most time in the Top 10
### Display the results in a Dataframe ordered by country name.

F_DF= filmsDF.groupBy('country_name', 'show_title').agg(F.max('cumulative_weeks_in_top_10').alias('max_weeks_in_top_10')).sort(F.desc('max_weeks_in_top_10'))
G_DF= filmsDF.groupBy('country_name').agg(F.max('cumulative_weeks_in_top_10').alias('max_weeks_in_top_10'))
d3 = F_DF.join(G_DF, on=["country_name", "max_weeks_in_top_10"], how="inner")
d3.show()

+------------+-------------------+--------------------+
|country_name|max_weeks_in_top_10|          show_title|
+------------+-------------------+--------------------+
|    Paraguay|                  9|  Sonic the Hedgehog|
|      Russia|                 16|          Red Notice|
|      Sweden|                  8|              Hustle|
|      Sweden|                  8|    The Adam Project|
| Philippines|                  7|          Red Notice|
|    Malaysia|                 14|Upin & Ipin: The ...|
|   Singapore|                  7|          Red Notice|
|      Turkey|                 13|          Red Notice|
|     Germany|                 13|Harry Potter and ...|
|      Jordan|                 14|          Red Notice|
|    Maldives|                 14|        Sooryavanshi|
|      France|                  6|    The Adam Project|
|      France|                  6|       Don't Look Up|
|      France|                  6|       The Sea Beast|
|      France|                  6|        The Ta

# Exercise 6

Calculate the number of weeks each film spent at the number 1 spot of each country's Top 10 list. Then find the films that spent the most time in the number 1 spot for each country.

In [192]:
# CodeGrade Tag Ex6a

### Create a column using the F.when function to calculate the number of weeks a
### films spens in the number 1 spot of the Top 10. Use the .otherwise method to
### set rows with no number 1 spots to zero. Use the .alias metod to call this
### column "weeks_at_1"


# Add a new column 'weeks_at_1' to the DataFrame
# Assuming there's a column 'top_spot_weeks' that keeps track of the number of weeks at number 1
# If there's no such column, you will need to define the logic on how to determine if a film was at number 1

netflixDF = netflixDF.withColumn(
    "weeks_at_1",
    F.when(F.col("top_spot_weeks") > 0, F.col("top_spot_weeks")).otherwise(0)
).alias("weeks_at_1")

# Show the DataFrame to verify the new column
netflixDF.show()



AnalysisException: ignored

In [ ]:
# CodeGrade Tag Ex6b

### Group by country name and sow title, and use the .agg method and your new
### column to find the number of weeks each film spent in the top spot for each
### country.



In [ ]:
# CodeGrade Tag Ex6c

### Produce a dataframe grouped by country name that contains the show title and
### number of weeks at the number 1 spot of the top performing film in each
### country.

